In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange
from numba import njit, prange

import torch
from torch import nn


from data import addition_dataset

source = "../source"
sys.path.append(source)

from preprocessing import OneHot, Encoding
from compilation import Compiler, ScalarTracker, ActivationTracker
from data_analysis.automata import to_automaton_history, reduce_automaton, to_automaton
from visualization.animation import SliderAnimation
from visualization.activations import ActivationsAnimation
from visualization.automata import AutomatonAnimation, display_automata
from visualization.epochs import EpochAnimation

import publication

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [2]:
symbols = [str(n) for n in range(10)] + ["+", " "]
onehot = OneHot(symbols)


class SequenceEncoder(Encoding):
    def __init__(self, single_encoder: Encoding):
        self._decoding = lambda x: x
        self.single_encoder = single_encoder

    # @property
    # def symbols(self) -> list:
    #     return None

    # def _update_decoding(self, encoding):
    #     pass

    def __call__(self, data):
        encoded = [self.single_encoder(char) for char in data]
        encoded = encoded[::-1]  # decoded in reverse for better computation
        return encoded

    def decode(self, enc_data):
        enc_data = enc_data.to("cpu").detach()
        decoded = [self.single_encoder.decode(vec) for vec in enc_data]
        decoded = "".join(decoded)
        decoded = decoded[::-1]
        return decoded


encoder = SequenceEncoder(onehot)

In [3]:
## Generate data

full_length = 100
max_train_length = 20
test_length = 30

test_data = addition_dataset(
    device, encoder, n_datapoints=100, seq_len=[test_length], full_length=full_length
)

In [4]:
## Define model


class Model(nn.Module):
    def __init__(self, device, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.lstm_encoder = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        self.lstm_decoder = nn.LSTM(
            input_size=hidden_size * 2,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        self.linear = nn.Linear(hidden_size * 2, output_size)

        self.to(device)

    def forward(self, x):
        x, h = self.lstm_encoder(x)
        x, _ = self.lstm_decoder(x)
        x = self.linear(x)
        x = torch.nn.functional.softmax(x, dim=-1)
        return x, h


model = Model(
    device,
    input_size=len(symbols),
    hidden_size=100,
    num_layers=1,
    output_size=len(symbols),
)

In [5]:
def predict(input, model):
    out = model(input)[0]
    pred = torch.argmax(out, axis=1).int()
    string = "".join([symbols[index] for index in pred])

    string = string[::-1]
    return string


def score(dataset, model):
    count = 0
    n_datapoints = len(dataset)
    for n in prange(n_datapoints):
        x, y = dataset[n]
        target = encoder.decode(y)
        prediction = predict(x, model)
        if target == prediction:
            count += 1
    acc = count / len(dataset)
    return acc

In [6]:
model = torch.load("model.pt")

In [7]:
train_losses, test_losses, train_scores, test_scores = [], [], [], []

In [8]:
## Train

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

n_epochs = 5000

for epoch in range(n_epochs):
    # Generate data
    train_data = addition_dataset(
        device,
        encoder,
        n_datapoints=1000,
        seq_len=[x for x in range(3, max_train_length) if x != test_length],
        full_length=full_length,
    )
    train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=32)

    # Validation
    if epoch % 50 == 0:
        model.eval()
        with torch.no_grad():
            y_pred = model(train_data.tensors[0])[0]
            train_rmse = np.sqrt(loss_fn(y_pred, train_data.tensors[1]).cpu())
            y_pred = model(test_data.tensors[0])[0]
            test_rmse = np.sqrt(loss_fn(y_pred, test_data.tensors[1]).cpu())
            train_score = score(
                torch.utils.data.Subset(train_data, np.arange(100)), model
            )
            test_score = score(test_data, model)
            train_losses.append(train_rmse)
            test_losses.append(test_rmse)
            train_scores.append(train_score)
            test_scores.append(test_score)
        print(
            f"Epoch {epoch}: train RMSE {train_rmse:.4f}, test RMSE {test_rmse:.4f}, train score {train_score:.2f}, test score {test_score:.2f}"
        )

    # Train
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)[0]
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch 0: train RMSE 0.2420, test RMSE 0.2444, train score 0.79, test score 0.32


KeyboardInterrupt: 

In [ ]:
# torch.save(model, "model.pt")

In [ ]:
## Plot loss curves
c = 1
# publication.set_color_mixed()
plt.plot(c * np.array(train_losses), label="Train loss")
plt.plot(c * np.array(test_losses), label="Test loss")
plt.plot(train_scores, label="Train score")
plt.plot(test_scores, label="Test score")
plt.legend()
# publication.plt_show()

In [28]:
int_a = 788778
int_b = 878787878
target = int_a + int_b
example = f"{int_a}+{int_b}"

print(f"Example length: {len(example)}")

computation_length = len(example) + full_length
example = example + " " * (computation_length - len(example))
encoded = torch.tensor(encoder(example), device=device, dtype=torch.float32)
prediction = predict(encoded, model)

print(f"input: {example}")
print(f"target: {target}")
print(f"prediction: {prediction}")
print("Correct!" if int(prediction) == target else "Wrong!")

Example length: 16
input: 788778+878787878                                                                                                    
target: 879576656
prediction: 789656666                                                                                                           
Wrong!


/tmp/ipykernel_3572487/3063543233.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716578970623/work/torch/csrc/utils/tensor_new.cpp:274.)
  encoded = torch.tensor(encoder(example), device=device, dtype=torch.float32)


In [ ]:
## Check for large sequence length generalization

# Evaluate model generalization for sequences of varying lengths
N = 100
val_scores = []
for n in trange(3, N + 1):
    val_data = addition_dataset(
        device, encoder, n_datapoints=10, seq_len=[n], full_length=full_length
    )
    val_score = score(val_data, model)
    val_scores.append(val_score)

# Visualize validation error for varying sequence lengths
# publication.set_color_mixed()
fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.bar(np.arange(3, N + 1), val_scores, color="skyblue")
ax.set_xlabel("Sequence lengths")
ax.set_ylabel("Validation error")
ax.set_title("Model Generalization Across Sequence Lengths")
ax.set_yticks(np.arange(0, 1, 0.1))
# publication.pub_show()

In [11]:
model(torch.empty())

TypeError: empty() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [29]:

model.encoding = onehot

initial_hidden = torch.zeros(size=(1,model.lstm_encoder.hidden_size))

to_automaton(model, train_data, initial_hidden)

ValueError: Shape of passed values is (2, 100), indices imply (100, 100)